In [ ]:
# %% Calculus 2 - Section 11.76
#    Code challenge: approximating curve lengths

# This code pertains to a calculus course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/pycalc2_x
# The code in this repository is developed to solve the exercises provided along
# the course, and it has been written partially indepentently and partially
# from the code developed by the course instructor.


In [1]:
import numpy             as np
import sympy             as sym
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import scipy.integrate   as spi
import math
import mpmath

from scipy.signal                     import find_peaks
from IPython.display                  import display,Math
from google.colab                     import files
from IPython.display                  import Audio
from scipy.io                         import wavfile
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')

import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')


In [ ]:
# %% Exercise 1
#    Implement the parametric function shown in the video, compute its
#    derivatives and plot the function with sympy

# Function
t = sym.symbols('t')
x = sym.sin(t) * (sym.exp(sym.cos(t)) - 2*sym.cos(4*t) - sym.sin(t/12)**5)
y = sym.cos(t) * (sym.exp(sym.cos(t)) - 2*sym.cos(4*t) - sym.sin(t/12)**5)

# Derivatives
dx = sym.diff(x,t)
dy = sym.diff(y,t)

# Print
display(Math('x(t) = %s' %sym.latex(x))), print()
display(Math("x'(t) = %s" %sym.latex(dx))), print('\n')

display(Math('y(t) = %s' %sym.latex(y))), print()
display(Math("y'(t) = %s" %sym.latex(dy)))


In [ ]:
# %% Exercise 1
#    continuing ...

# Bounds
a = 0
b = 2*sym.pi

# Plot
p = sym.plot_parametric(x,y,(t,a,b),title='A parametric function',ylabel='y(t)',xlabel='x(t)',legend=True,label='function',show=False);
p._backend.process_series()
lines = p._backend.ax.get_lines()
lines[0].set_linewidth(2)

p._backend.fig.savefig('fig18_codechallenge_76_exercise_1.png')
files.download('fig18_codechallenge_76_exercise_1.png')


In [ ]:
# %% Exercise 2
#    Use spi.quad() to approximate the length of the curve

# Integrand
integrand = sym.lambdify( t,sym.sqrt( dx**2+dy**2 ) )

# and now to integrate
length,_  = spi.quad(integrand,a,b)

print(f'Curve length using spi.quad: {length}')


In [ ]:
# %% Exercise 3
#    Lambdify and visualise the function and the derivatives, both implicitly
#    and explicitly

x_l  = sym.lambdify(t,x)
y_l  = sym.lambdify(t,y)

dx_l = sym.lambdify(t,dx)
dy_l = sym.lambdify(t,dy)

tt = np.linspace(float(a),float(b),999)

phi = (1 + np.sqrt(5)) / 2
_,axs = plt.subplots(2,2,figsize=(phi*6,6))

axs[0,0].plot(x_l(tt),y_l(tt),'tab:red')
axs[0,0].set(xlabel='$x$',ylabel='$y$',title='Implicit parametric function')

axs[0,1].plot(tt,x_l(tt),label='x(t)')
axs[0,1].plot(tt,y_l(tt),label='y(t)')
axs[0,1].set(xlim=tt[[0,-1]],xlabel='$t$',ylabel='$x$ or $y$',title='Explicit parametric functions')
axs[0,1].legend()

axs[1,0].plot(dx_l(tt),dy_l(tt),'tab:green')
axs[1,0].set(xlabel='$dx$',ylabel='$dy$',title='Derivative')

axs[1,1].plot(tt,dx_l(tt),label="x'(t)")
axs[1,1].plot(tt,dy_l(tt),label="y'(t)")
axs[1,1].set(xlim=tt[[0,-1]],xlabel='$t$',ylabel='$dx$ or $dy$',title='Derivatives')
axs[1,1].legend()

plt.tight_layout()

plt.savefig('fig19_codechallenge_76_exercise_3.png')
plt.show()
files.download('fig19_codechallenge_76_exercise_3.png')


In [ ]:
# %% Exercise 4
#    Explore the effect of resolution over the approximation accuracy

# Samples
Ns = np.linspace(10,1000,40).astype(int)

# Preallocate
lengths_npy  = np.zeros(len(Ns))
lengths_spi  = np.zeros(len(Ns))

# Initialise plotting and compute
phi = (1 + np.sqrt(5)) / 2
_,axs = plt.subplots(1,2,figsize=(1.5*phi*5,5))

# loop over the values of dt
for idx,n in enumerate(Ns):

  ttt = np.linspace(float(a),float(b),n)
  dt  = ttt[1] - ttt[0]

  # Numpy solution
  dx_squared       = (dx_l(ttt)*dt)**2
  dy_squared       = (dy_l(ttt)*dt)**2
  lengths_npy[idx] = np.sum( np.sqrt( dx_squared + dy_squared ) )

  # Simpson's rule
  integrand        = np.sqrt((dx_l(ttt))**2 + (dy_l(ttt))**2)
  lengths_spi[idx] = spi.simpson(integrand,dx=dt)

  # Plot first and last curves
  if idx==0:
    axs[0].plot(x_l(ttt),y_l(ttt),'-',linewidth=2,label='N = %g'%n)
  elif idx==(len(Ns)-1):
    axs[0].plot(x_l(ttt),y_l(ttt),'-',linewidth=2,label='N = %g'%n)

# Finalise plotting
axs[0].set(xlabel='$x(t)$',ylabel='$y(t)$',title='Example curves for N = {10,1000}')
axs[0].legend()

axs[1].plot(Ns,lengths_npy,'s-',label='NumPy')
axs[1].plot(Ns,lengths_spi,'s-',label="Simpson's")
axs[1].axhline(length,color='k',linestyle='--',label='spi.quad')
axs[1].set(xlabel='N',ylabel='Curve length',title='Curve length approximation by N')
axs[1].legend()

plt.tight_layout()

plt.savefig('fig20_codechallenge_76_exercise_4.png')
plt.show()
files.download('fig20_codechallenge_76_exercise_4.png')
